# Learning and Decision Making

## Laboratory 3: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the knight domain described in the Homework and which you described as a Markov decision process.

<img src="knight.png" width="200px">

Recall that:

* At each step, the knight may move in any of the four directions---up, down, left and right. 

* The movement succeeds with a 0.6 probability and fails with a 0.4 probability. When the movement fails, the knight may stay in the same cell or move to one of the immediately adjacent cells (if there is one) with equal probability.

* The goal of the knight is to save (reach) the princess and avoid the dragon.

**Throughout the lab, use $\gamma=0.99$.**

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs. Make sure that:
    * The costs lie in the interval [0, 1]
    * The cost for standing in the princess's cell is minimal
    * The cost for standing in the dragon's cell is maximal
    * The costs for the intermediate cells are around 1/5 of those of standing in the dragon's cell

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note**: Don't forget to import `numpy`.

---

In [1]:
import numpy as np

X = [0,1,2,3,4,5]

#0 corresponds to cell 1
#1 corresponds to cell 2
#2 corresponds to cell 3
#3 corresponds to cell 4
#4 corresponds to cell 5
#5 corresponds to cell 6

A = ['LEFT','RIGHT','UP','DOWN']


#Transition probabilities matrix for L
pmL = np.array([[0.8,0.1,0,0.1,0,0],
                [0.6,0.2,0.1,0,0.1,0],
                [0,0.6,0.3,0,0,0.1],
                [0.1,0,0,0.8,0.1,0],
                [0,0.1,0,0.6,0.2,0.1],
                [0,0,0.1,0,0.6,0.3]])

#Transition probabilities matrix for R
pmR = np.array([[0.3,0.6,0,0.1,0,0],
                [0.1,0.2,0.6,0,0.1,0],
                [0,0.1,0.8,0,0,0.1],
                [0.1,0,0,0.3,0.6,0],
                [0,0.1,0,0.1,0.2,0.6],
                [0,0,0.1,0,0.1,0.8]])

#Transition probabilities matrix for U
pmU = np.array([[0.8,0.1,0,0.1,0,0],
                [0.1,0.7,0.1,0,0.1,0],
                [0,0.1,0.8,0,0,0.1],
                [0.6,0,0,0.3,0.1,0],
                [0,0.6,0,0.1,0.2,0.1],
                [0,0,0.6,0,0.1,0.3]])

#Transition probabilities matrix for D
pmD = np.array([[0.3,0.1,0,0.6,0,0],
                [0.1,0.2,0.1,0,0.6,0],
                [0,0.1,0.3,0,0,0.6],
                [0.1,0,0,0.8,0.1,0],
                [0,0.1,0,0.1,0.7,0.1],
                [0,0,0.1,0,0.1,0.8]])


#Cost Function
cost_function = np.array([[0.2, 0.2, 0.2, 0.2],
                         [0.2, 0.2, 0.2, 0.2],
                         [0.2, 0.2, 0.2, 0.2],
                         [0.2, 0.2, 0.2, 0.2],
                         [1, 1, 1, 1],
                         [0, 0, 0, 0]])

### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $x$, always moves the knight to the cell closest to the princess (irrespectively of the dragon). If multiple such cells exist, the knight should select randomly between the two.

For example, suppose that the knight is in cell 1. The knight should then select randomly between the actions _D_ and _R_. Conversely, suppose that the knight is in cell 4. The knight should then select actions _R_ with probability 1.

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry _xa_ has the probability of selecting action _a_ in state _x_.

---

In [2]:
my_policy = np.array([[0, 0.5, 0, 0.5],
                      [0, 0.5, 0, 0.5],
                      [0, 0, 0, 1],
                      [0, 1, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0.5, 0, 0.5]])

---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2.

---

In [3]:
from numpy.linalg import linalg

identity_matrix = np.eye(6)
inflation = 0.99
cpi = (cost_function * my_policy).sum(axis=1) #cost_function[:,0] * my_policy[:,0] + ..
ppi = my_policy[:,0,None]*pmL + my_policy[:,1,None]*pmR + my_policy[:,2,None]*pmU + my_policy[:,3,None]*pmD

aux1 = ppi*inflation
aux2 = np.subtract(identity_matrix,aux1)
aux3 = linalg.inv(aux2)
cost_to_go = np.reshape(np.dot(aux3,cpi),(6,1))

print(cost_to_go)

[[16.26056701]
 [15.95826371]
 [15.28584405]
 [16.45495016]
 [16.42766638]
 [15.09441121]]


### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is _not_ optimal: use value iteration to compute $J^*$ and show that $J^*\neq J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [4]:
import time

i = 0
error = 1
cost_left = np.reshape(cost_function[:,0],(6,1))
cost_right = np.reshape(cost_function[:,1],(6,1))
cost_up = np.reshape(cost_function[:,2],(6,1))
cost_down = np.reshape(cost_function[:,3],(6,1))
J = np.zeros((6,1))
t1 = time.time()

while error > 1e-8:
    Ql = cost_left + inflation * pmL.dot(J)
    Qr = cost_right + inflation * pmR.dot(J)
    Qu = cost_up + inflation * pmU.dot(J)
    Qd = cost_down + inflation * pmD.dot(J)
    
    Jnew = np.min((Ql, Qr, Qu, Qd), axis=0)
    error = np.linalg.norm(Jnew - J)
    J = Jnew
    i += 1
    
t2 = time.time()

print("Time: " + str(t2-t1) + 's')
print("Iterations: " + str(i))
print(J)
print("In any state, the cost to go values are always less than the ones calculated previously, proving \
that the policy is not optimal")
    

Time: 0.043898582458496094s
Iterations: 1726
[[14.06797074]
 [13.93809867]
 [13.67954678]
 [14.24800834]
 [14.74722471]
 [13.53004987]]
In any state, the cost to go values are always less than the ones calculated previously, proving that the policy is not optimal


---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [5]:
import time

i = 0
pi = np.ones((6,4)) / 4
t1 = time.time()
quit = False

while not quit:
    cpi = np.diag(pi[:,0]).dot(cost_left) + np.diag(pi[:,1]).dot(cost_right) + np.diag(pi[:,2]).dot(cost_up) + np.diag(pi[:,3]).dot(cost_down) 
    ppi = np.diag(pi[:,0]).dot(pmL) + np.diag(pi[:,1]).dot(pmR) + np.diag(pi[:,2]).dot(pmU) + np.diag(pi[:,3]).dot(pmD) 
    J = np.linalg.inv(np.eye(6) - inflation * ppi).dot(cpi)
    
    Ql = cost_left + inflation * pmL.dot(J)
    Qr = cost_right + inflation * pmR.dot(J)
    Qu = cost_up + inflation * pmU.dot(J)
    Qd = cost_down + inflation * pmD.dot(J)
    
    pinew = np.zeros((6,4))
    pinew[:,0,None] = np.isclose(Ql,np.min([Ql,Qr,Qu,Qd], axis = 0), atol = 1e-8, rtol = 1e-8).astype(int)
    pinew[:,1,None] = np.isclose(Qr,np.min([Ql,Qr,Qu,Qd], axis = 0), atol = 1e-8, rtol = 1e-8).astype(int)
    pinew[:,2,None] = np.isclose(Qu,np.min([Ql,Qr,Qu,Qd], axis = 0), atol = 1e-8, rtol = 1e-8).astype(int)
    pinew[:,3,None] = np.isclose(Qd,np.min([Ql,Qr,Qu,Qd], axis = 0), atol = 1e-8, rtol = 1e-8).astype(int)
    pinew = pinew / np.sum(pinew, axis=1,keepdims = True)
    
    quit = (pi == pinew).all()
    pi = pinew
    i += 1
    
t2 = time.time()

print("Time: " + str(t2-t1) + 's')
print("Iterations: " + str(i))
print(pi)
    

Time: 0.0024123191833496094s
Iterations: 3
[[0.  1.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  1. ]
 [0.  0.  1.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.5 0.  0.5]]


### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Starting both in cell 1 and cell 5 in the figure, 

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4 for the two states. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [ ]:
from numpy import random as rand

p = {0 : pmL, 1 : pmR, 2 : pmU, 3 : pmD}
cells = [0,4]
costs = []
A = np.arange(0, 4)

for cell in cells:
    for trajectory in range(0,100):
        cost = 0
        inflation = 1
        current_cell = cell
        for step in range(0,10000):
            random_choice = rand.choice(A, p = pi[current_cell, :])
            prob = p[random_choice]
            cost += cost_function[current_cell][random_choice] * inflation
            inflation *= 0.99
            nextState = rand.choice(X, p = prob[current_cell])
            current_cell = nextState
        costs.append(cost)
    print("Starting on state", cell + 1, " the cost was =", np.sum(costs)/100)
    costs = []

print("Comparing the result with that computed in Activity 4 the values are approximately equal")

Starting on state 1  the cost was = 14.304002435636463
